# Replication of Paper Results

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import csv
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d

In [ ]:
import numpy as np

def create_multi_agent_dynamics_with_center(N, masses, L, controller, V_d, r, theta_d,
                                            k=2.5e4, lambda_i=1, g=9.81, lambda_theta=1):
    """
    """
    assert N >= 1, "There must be at least one agent (the center)."
    assert len(masses) == N

    e3 = np.array([0, 0, 1]).reshape(-1,1)



    def dynamics(t, state):
        dstate_dt = np.zeros_like(state)

        vd = V_d(t)
        u = controller(t, state, vd)

        

        for i in range(0, N):
            idx = 6 * i
            x_i = state[idx : idx + 3]
            v_i = state[idx + 3 : idx + 6]

            m_i = masses[i]
            F_i = u[:,i]
    

            dx_dt = v_i
            dv_dt = (F_i.reshape(-1,1) - m_i * g * e3) / m_i

            dstate_dt[idx : idx + 3] = dx_dt
            dstate_dt[idx + 3 : idx + 6] = dv_dt.flatten()

        for i in range(6*N, 6*N+N):
            item = i - 6*N
            dMc_i = -lambda_i*(state[item*6+5]-vd[2])+lambda_i*np.sum([state[6*N+j]-state[6*N+item] for j in range(0, N) if L[item, j]!=0])#
            dstate_dt[i] = dMc_i

        for i in range(7*N, 7*N+N):
            item = i - 7*N
            vec = np.zeros((3,1))
            vec[0] = -np.sin((state[i] + theta_d[i-7*N])/2)
            vec[1] = np.cos((state[i] + theta_d[i-7*N])/2)
            #print(state[6*item+3:6*item+6].shape)
            d_theta_i = -lambda_theta*np.sin((state[i]-theta_d[i-7*N])/2) - 2*controller.rho*(state[6*item+3:6*item+6].reshape(-1,1)-vd).T@vec
            dstate_dt[i] = d_theta_i

        
            
        return dstate_dt

    return dynamics

In [ ]:
class Controller():
    def __init__(self, N, Gamma_i, V_d, r, masses, g=9.81, rho=25, log_file='control_log.csv', f_log_file='f_hat_log.csv'):
        self.N = N
        self.Gamma_i = Gamma_i
        self.V_d = V_d
        self.masses = masses
        self.g = g
        self.log_file = log_file
        self.f_log_file = f_log_file
        self.r_norm = r/np.linalg.norm(r, axis=0, keepdims=True)
        self.rho = rho
        print(self.r_norm)

        # Create or overwrite the log file and write header
        with open(self.log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            header = ['time']
            for i in range(N):
                header += [f'u{i}_x', f'u{i}_y', f'u{i}_z']
            writer.writerow(header)
        
        # Create or overwrite the force log file and write header
        with open(self.f_log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            header = ['time']
            for i in range(N):
                header += [f'f{i}_x', f'f{i}_y', f'f{i}_z']
            writer.writerow(header)

    def __call__(self, t, state, vd):
        e3 = np.array([0, 0, 1]).reshape(-1,1)
        out = np.zeros((3, self.N))
        f_log = np.zeros((3, self.N))
        f_coeff = np.array([[0.0, -2.0], [2.0, 1.0], [-2.0, 1.0]])

        
        for i in range(self.N):
            f_hat_i = np.zeros((3,1))
            f_hat_i[0] = (1-np.exp(-t))*np.cos(state[7*N + i])*self.rho
            f_hat_i[1] = (1-np.exp(-t))*np.sin(state[7*N + i])*self.rho
            f_hat_i[2] = (1-np.exp(-t))*state[6*(self.N)+i]*self.g

            f_log[:, i] = f_hat_i.flatten()
            
            out[:,i] = (f_hat_i + self.masses[i]*self.g*e3 - self.Gamma_i @ (state[(i)*6+3:(i)*6+6].reshape(-1,1) - vd)).flatten() 

        # Log control input to CSV
        flat_output = out.T.flatten()  # [u0_x, u0_y, u0_z, u1_x, ...]
        with open(self.log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([t] + flat_output.tolist())
        
        # Log force estimates
        with open(self.f_log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([t] + f_log.T.flatten().tolist())

        return out


In [ ]:
class CircularCloth:
    def __init__(self,
                 radius=2.5,
                 num_rings=10,
                 points_per_ring=30,
                 k=500.0,
                 m=0.1,
                 damping=0.1,
                 gravity=np.array([0, 0, -9.81]),
                 num_beams=3,
                 beam_length=1.5,
                 beam_nodes_per_beam=1,
                 central_index=0):
        
        self.radius = radius
        self.num_rings = num_rings
        self.points_per_ring = points_per_ring
        self.k = k
        self.m = m
        self.damping = damping
        self.gravity = gravity
        self.central_index = central_index
        
        self.num_beams = num_beams
        self.beam_length = beam_length
        self.beam_nodes_per_beam = beam_nodes_per_beam
        
        self.nodes = None
        self.N = 0
        self.edges = []
        self.rest_lengths = None
        self.masses = None
        self.fixed_indices = []
        self.beam_edges = []
        self.state0 = None
        self.adjacency_matrix = None
        
        self._generate_nodes()
        self._generate_edges_and_rest_lengths()
        #self._add_beams()
        self._finalize_state()
        
    def _generate_nodes(self):
        nodes = []
        nodes.append(np.array([0, 0, 0]))  # center point
        for r in range(1, self.num_rings):
            radius_r = self.radius * r / (self.num_rings - 1)
            for i in range(self.points_per_ring):
                angle = 2 * np.pi * i / self.points_per_ring
                x = radius_r * np.cos(angle)
                y = radius_r * np.sin(angle)
                nodes.append(np.array([x, y, 0]))
        self.nodes = np.array(nodes)
        self.N = len(self.nodes)

        # Salva gli indici equispaziati dell'anello più esterno
        start_idx = 1 + (self.num_rings - 2) * self.points_per_ring
        end_idx = start_idx + self.points_per_ring

        # Seleziona N indici equidispaziati tra i nodi dell’anello esterno
        total = self.points_per_ring
        # o self.N_target se definito altrove
        step = total // self.num_beams

        self.fixed_indices = [start_idx + i * step for i in range(self.num_beams)]
        #print(f"Outer ring indices: {self.outer_ring_indices}")

        
    def _get_neighbors(self, idx):
        if idx == 0:
            return list(range(1, self.points_per_ring + 1))
        
        ring_idx = None
        count = 1
        for r in range(1, self.num_rings):
            if idx < count + self.points_per_ring:
                ring_idx = r
                break
            count += self.points_per_ring

        pos_in_ring = idx - (1 + (ring_idx - 1) * self.points_per_ring)

        neighbors = []
        neighbors.append(1 + (ring_idx - 1) * self.points_per_ring + (pos_in_ring - 1) % self.points_per_ring)
        neighbors.append(1 + (ring_idx - 1) * self.points_per_ring + (pos_in_ring + 1) % self.points_per_ring)

        if ring_idx > 1:
            inner_ring_start = 1 + (ring_idx - 2) * self.points_per_ring
            inner_pos = int(pos_in_ring * self.points_per_ring / self.points_per_ring)
            neighbors.append(inner_ring_start + inner_pos % self.points_per_ring)
            neighbors.append(inner_ring_start + (inner_pos - 1) % self.points_per_ring)
        else:
            neighbors.append(0)
        
        if ring_idx < self.num_rings - 1:
            outer_ring_start = 1 + ring_idx * self.points_per_ring
            neighbors.append(outer_ring_start + pos_in_ring % self.points_per_ring)
            neighbors.append(outer_ring_start + (pos_in_ring + 1) % self.points_per_ring)

        neighbors = list(set(n for n in neighbors if 0 <= n < self.N))
        return neighbors
    
    def _generate_edges_and_rest_lengths(self):
        edges = []
        rest_lengths = []
        for i in range(self.N):
            for j in self._get_neighbors(i):
                if i < j:
                    edges.append((i, j))
                    dist = np.linalg.norm(self.nodes[i] - self.nodes[j])
                    rest_lengths.append(dist)
        self.edges = edges
        self.rest_lengths = np.array(rest_lengths)
    
    """def _add_beams(self):
        beam_nodes = []
        beam_edges = []

        beam_angles = np.linspace(0, 2*np.pi, self.num_beams, endpoint=False)
        outer_ring_start = 1 + (self.num_rings - 2) * self.points_per_ring
        outer_ring_end = outer_ring_start + self.points_per_ring

        for angle in beam_angles:
            outer_ring_nodes = self.nodes[outer_ring_start:outer_ring_end]
            target_pos = np.array([self.radius * np.cos(angle), self.radius * np.sin(angle), 0])
            dists = np.linalg.norm(outer_ring_nodes - target_pos, axis=1)
            closest_idx = np.argmin(dists) + outer_ring_start

            prev_idx = closest_idx
            beam_nodes.append(prev_idx)  # fixed outer ring node connected to beam

            for i in range(1, self.beam_nodes_per_beam):
                dist_out = self.radius + i * self.beam_length / (self.beam_nodes_per_beam - 1)
                x = dist_out * np.cos(angle)
                y = dist_out * np.sin(angle)
                z = 0.0
                new_node = np.array([x, y, z])
                self.nodes = np.vstack([self.nodes, new_node])
                new_idx = self.N
                beam_nodes.append(new_idx)
                beam_edges.append((prev_idx, new_idx))
                prev_idx = new_idx
                self.N += 1

        self.beam_edges = beam_edges
        self.edges.extend(beam_edges)

        for (i, j) in beam_edges:
            dist = np.linalg.norm(self.nodes[i] - self.nodes[j])
            self.rest_lengths = np.append(self.rest_lengths, dist)

        self.fixed_indices = list(set(beam_nodes))
        print(f"Fixed indices for beams: {self.fixed_indices}") """
    
    def _finalize_state(self):
        self.masses = np.full(self.N, self.m)
        self.state0 = np.concatenate((self.nodes.flatten(), np.zeros(3 * self.N)))

        self.adjacency_matrix = np.zeros((self.N, self.N))
        for i, j in self.edges:
            self.adjacency_matrix[i, j] = 1
            self.adjacency_matrix[j, i] = 1
        print(f"Adjacency matrix:\n{self.adjacency_matrix}")
        
    
    def compute_forces(self, positions, velocities):
        forces = np.zeros_like(positions)
        for i in range(self.N):
            forces[3*i:3*i+3] =  forces[3*i:3*i+3] + self.gravity.reshape(3) * float(self.masses[i])
        
        for idx, (i, j) in enumerate(self.edges):
            pos_i = positions[3*i:3*i+3]
            pos_j = positions[3*j:3*j+3]
            delta = pos_i - pos_j
            dist = np.linalg.norm(delta)
            if dist == 0:
                continue
            direction = delta / dist
            force_magnitude = -self.k * (dist - self.rest_lengths[idx])
            force = force_magnitude * direction
            forces[3*i:3*i+3] += force
            forces[3*j:3*j+3] -= force
        
        for i in range(self.N):
            vel = velocities[3*i:3*i+3]
            forces[3*i:3*i+3] -= self.damping * vel
        
        
        return forces
    
    def ode(self, t, state):
        positions = state[:3*self.N]
        velocities = state[3*self.N:]

        forces = self.compute_forces(positions, velocities)
        accelerations = forces / np.repeat(self.masses, 3)
        
        derivatives = np.zeros_like(state)
        derivatives[:3*self.N] = velocities
        derivatives[3*self.N:] = accelerations
        
    
        
        return derivatives


In [ ]:

def total_dynamics(N, m_drone, V_d, Gamma_i, masses, L, r, k, lambda_i, m_ball, v_ball, t_ball, theta_d, controller, cloth, log_file='force_log.csv'):
     # Initialize the log file
    if os.path.exists(log_file):
        os.remove(log_file)
    with open(log_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header = ['time'] + [f'f{i}_{axis}' for i in range(N) for axis in ['x', 'y', 'z']]
        writer.writerow(header)
    def dynamics(t, state):

        if np.abs(t-t_ball) < 1e-2 and flag_ball[0] == False:
            cloth.masses[cloth.central_index] = cloth.m + m_ball
            state[8*N + 3*cloth.N + 3*cloth.central_index + 2] = (cloth.m*state[8*N + 3*cloth.N + 3*cloth.central_index + 2] + m_ball*V_d(t)[2])/(cloth.m + m_ball)
            flag_ball[0] = True
            print(f"Ball added at time {t}")

        print(f"t: {t:.4f}", end="\r", flush=True)

        drones_state = state[:8*N]
        drones_dynamics = create_multi_agent_dynamics_with_center(N, masses, L, controller, V_d, r, theta_d=theta_d, lambda_i = lambda_i)
        drones_derivatives = drones_dynamics(t, drones_state)

        cloth_state = state[8*N:]
        cloth_derivatives = cloth.ode(t, cloth_state)

        # Interface physics 
        #fixed_indices = [idx + 7*N for idx in cloth.fixed_indices]

        f = np.zeros((3, N))
        #Check che al primo drone corrisponda il primo punto nella lista
        for i, idx in enumerate(cloth.fixed_indices):
            x_i_pos = drones_state[6*i:6*i+3]

            a_i_pos = cloth_state[3*idx:3*idx+3]

            f_i = k*(x_i_pos - a_i_pos)
            f[:, i] = f_i.flatten()
            drones_derivatives[6*i+3:6*i+6] = drones_derivatives[6*i+3:6*i+6] - f_i/masses[i]
            cloth_derivatives[3*cloth.N + 3*idx:3*cloth.N + 3*idx+3] = cloth_derivatives[3*cloth.N + 3*idx:3*cloth.N + 3*idx+3] + f_i/cloth.m
            #drones_derivatives[N*i+3:N*i+6] = cloth_derivatives[3*cloth.N + 3*idx:3*cloth.N + 3*idx+3] #drones_derivatives[N*i+3:N*i+6] - f_i/masses[i]
            #drones_derivatives[&*i:N*i+3] = cloth_state[3*idx:3*idx+3]

        # Log time and f_i
        with open(log_file, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            flat_f = f.T.flatten()  # shape: (3*N,) in [f1x, f1y, f1z, f2x, ...]
            writer.writerow([t] + flat_f.tolist())

        return np.concatenate([drones_derivatives, cloth_derivatives])

    return dynamics
    

In [ ]:
def compute_laplacian_matrix(N, type='full'):
	L = np.zeros((N,N))
	if type == 'full':
		L = -np.ones((N,N)) + np.diag(N*np.ones(N))
	elif type == 'neighbors':
		L = np.diag(2*np.ones(N)) - np.diag(np.ones(N-1), k = 1) - np.diag(np.ones(N-1), k = -1)
		L[0, -1] = -1
		L[-1, 0] = -1
	return L

In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# --- Polynomial coefficient functions (same as before) ---
def compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1):
    T = t1
    zcf = zc0 + vc0*T + 0.5*ac0*T**2
    vcf = vc0 + ac0*T
    acf = ac0
    A = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [1, T, T**2, T**3, T**4, T**5],
        [0, 1, 2*T, 3*T**2, 4*T**3, 5*T**4],
        [0, 0, 2, 6*T, 12*T**2, 20*T**3]
    ])
    b = np.array([z0, vz0, az0, zcf, vcf, acf])
    return np.linalg.solve(A, b)

def compute_coeffs_phase2(coeffs1, t1, t2):
    coeffs1_rev = coeffs1[::-1]
    j_poly = np.polyder(coeffs1_rev, 3)
    j_end = np.polyval(j_poly, t1)
    z_end = np.polyval(coeffs1_rev, t1)
    vz_end = np.polyval(np.polyder(coeffs1_rev, 1), t1)
    az_end = np.polyval(np.polyder(coeffs1_rev, 2), t1)
    T = t2
    A = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 0, 6, 0, 0],
        [0, 1, 2*T, 3*T**2, 4*T**3, 5*T**4],
        [0, 0, 2, 6*T, 12*T**2, 20*T**3]
    ])
    b = np.array([z_end, vz_end, az_end, j_end, 0, 0])
    return np.linalg.solve(A, b)

# --- Constraints helper functions ---
def make_acc_upper_constraint(phase, idx, z0, vz0, az0, zc0, vc0, ac0, N, limit=9.81):
    def constraint(x):
        t1, t2 = x
        if t1 <= 0 or t2 <= 0:
            return -1e12
        if phase == 1:
            coeffs = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
            t_vals = np.linspace(0, t1, N)
        else:
            coeffs1 = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
            coeffs = compute_coeffs_phase2(coeffs1, t1, t2)
            t_vals = np.linspace(0, t2, N)
        az = np.polyval(np.polyder(coeffs[::-1], 2), t_vals)
        return limit - az[idx]
    return constraint

def make_acc_lower_constraint(phase, idx, z0, vz0, az0, zc0, vc0, ac0, N, limit=-2*9.81):
    def constraint(x):
        t1, t2 = x
        if t1 <= 0 or t2 <= 0:
            return -1e12
        if phase == 1:
            coeffs = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
            t_vals = np.linspace(0, t1, N)
        else:
            coeffs1 = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
            coeffs = compute_coeffs_phase2(coeffs1, t1, t2)
            t_vals = np.linspace(0, t2, N)
        az = np.polyval(np.polyder(coeffs[::-1], 2), t_vals)
        return az[idx] - limit
    return constraint

def make_pos_constraint(phase, idx, z0, vz0, az0, zc0, vc0, ac0, z_min, N):
    def constraint(x):
        t1, t2 = x
        if t1 <= 0 or t2 <= 0:
            return -1e12
        if phase == 1:
            coeffs = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
            t_vals = np.linspace(0, t1, N)
        else:
            coeffs1 = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
            coeffs = compute_coeffs_phase2(coeffs1, t1, t2)
            t_vals = np.linspace(0, t2, N)
        z = np.polyval(coeffs[::-1], t_vals)
        return z[idx] - z_min
    return constraint


# --- Objective function ---
def objective_with_penalty(x, z0, vz0, az0, zc0, vc0, ac0, z_min, N=50, jerk_weight=1.0):
    t1, t2 = x
    if t1 <= 0 or t2 <= 0:
        return 1e6

    # Phase 1 trajectory
    coeffs1 = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
    t_vals1 = np.linspace(0, t1, N)
    z1 = np.polyval(coeffs1[::-1], t_vals1)
    if np.any(z1 < z_min):
        return 1e6

    jerk1 = np.polyval(np.polyder(coeffs1[::-1], 3), t_vals1)
    jerk_penalty1 = np.trapz(jerk1**2, t_vals1)

    # Phase 2 trajectory
    coeffs2 = compute_coeffs_phase2(coeffs1, t1, t2)
    t_vals2 = np.linspace(0, t2, N)
    z2 = np.polyval(coeffs2[::-1], t_vals2)
    if np.any(z2 < z_min):
        return 1e6

    jerk2 = np.polyval(np.polyder(coeffs2[::-1], 3), t_vals2)
    jerk_penalty2 = np.trapz(jerk2**2, t_vals2)

    # Total objective: time + jerk penalty
    jerk_penalty = jerk_weight * (jerk_penalty1 + jerk_penalty2)
    return jerk_penalty


# --- Main optimization function ---
def optimize_times_nonlinear(z0, vz0, az0, zc0, vc0, ac0, z_min):
    x0 = np.array([1.0, 1.0])  # initial guess
    
    bounds = [(0.1, None), (0.1, None)]  # t1,t2 > 0.1s
    
    N_sample = 100
    
    constraints = []

    for i in range(N_sample):
        constraints.append({
            'type': 'ineq',
            'fun': make_acc_upper_constraint(1, i, z0, vz0, az0, zc0, vc0, ac0, N_sample)
        })
        constraints.append({
            'type': 'ineq',
            'fun': make_acc_lower_constraint(1, i, z0, vz0, az0, zc0, vc0, ac0, N_sample)
        })
        constraints.append({
            'type': 'ineq',
            'fun': make_pos_constraint(1, i, z0, vz0, az0, zc0, vc0, ac0, z_min, N_sample)
        })

    for i in range(N_sample):
        constraints.append({
            'type': 'ineq',
            'fun': make_acc_upper_constraint(2, i, z0, vz0, az0, zc0, vc0, ac0, N_sample)
        })
        constraints.append({
            'type': 'ineq',
            'fun': make_acc_lower_constraint(2, i, z0, vz0, az0, zc0, vc0, ac0, N_sample)
        })
        constraints.append({
            'type': 'ineq',
            'fun': make_pos_constraint(2, i, z0, vz0, az0, zc0, vc0, ac0, z_min, N_sample)
        })

    
    res = minimize(
        objective_with_penalty, 
        x0, 
        args=(z0, vz0, az0, zc0, vc0, ac0, z_min), 
        bounds=bounds, 
        constraints=constraints,
        method='SLSQP', 
        options={
            'disp': True, 
            'maxiter': 500,
            'ftol': 1e-9,   # tighter solution tolerance
            'eps': 1e-6     # finer finite-difference step for better constraint satisfaction
        }
    )
    
    return res.x

# --- Plot function (reuse your existing code) ---
def compute_chaser_trajectory(z0, vz0, az0, zc0, vc0, ac0, t1, t2, z_min, N=500, plot=True):
    coeffs1 = compute_coeffs_phase1(z0, vz0, az0, zc0, vc0, ac0, t1)
    coeffs2 = compute_coeffs_phase2(coeffs1, t1, t2)
    
    t_phase1 = np.linspace(0, t1, N)
    t_phase2 = np.linspace(0, t2, N)
    
    # Ball only up to t1
    t_ball = np.linspace(0, t1, N)
    
    # Chaser phases
    z_phase1 = np.polyval(coeffs1[::-1], t_phase1)
    vz_phase1 = np.polyval(np.polyder(coeffs1[::-1], 1), t_phase1)
    az_phase1 = np.polyval(np.polyder(coeffs1[::-1], 2), t_phase1)
    j_phase1 = np.polyval(np.polyder(coeffs1[::-1], 3), t_phase1)
    
    z_phase2 = np.polyval(coeffs2[::-1], t_phase2)
    vz_phase2 = np.polyval(np.polyder(coeffs2[::-1], 1), t_phase2)
    az_phase2 = np.polyval(np.polyder(coeffs2[::-1], 2), t_phase2)
    j_phase2 = np.polyval(np.polyder(coeffs2[::-1], 3), t_phase2)
    
    # Ball trajectory until t1
    z_ball = zc0 + vc0 * t_ball + 0.5 * ac0 * t_ball**2
    vz_ball = vc0 + ac0 * t_ball
    az_ball = np.full_like(t_ball, ac0)
    
    if plot:
        fig, axs = plt.subplots(4, 1, figsize=(10, 12), sharex=True)
        
        axs[0].plot(t_phase1, z_phase1, label='Chaser Phase 1')
        axs[0].plot(t_phase2 + t1, z_phase2, label='Chaser Phase 2')
        axs[0].plot(t_ball, z_ball, '--', color='k', label='Ball (until t1)')
        axs[0].axhline(z_min, color='r', linestyle='--', label='z_min')
        axs[0].set_ylabel('Position z (m)')
        axs[0].legend()
        axs[0].grid()
        
        axs[1].plot(t_phase1, vz_phase1, label='Chaser Phase 1')
        axs[1].plot(t_phase2 + t1, vz_phase2, label='Chaser Phase 2')
        axs[1].plot(t_ball, vz_ball, '--', color='k', label='Ball (until t1)')
        axs[1].set_ylabel('Velocity vz (m/s)')
        axs[1].legend()
        axs[1].grid()
        
        axs[2].plot(t_phase1, az_phase1, label='Chaser Phase 1')
        axs[2].plot(t_phase2 + t1, az_phase2, label='Chaser Phase 2')
        axs[2].plot(t_ball, az_ball, '--', color='k', label='Ball (until t1)')
        axs[2].axhline(9.81, color='r', linestyle='--', label='Max Acceleration')
        axs[2].axhline(-2*9.81, color='r', linestyle='--', label='Min Acceleration')
        axs[2].set_ylabel('Acceleration az (m/s²)')
        axs[2].legend()
        axs[2].grid()
        
        axs[3].plot(t_phase1, j_phase1, label='Chaser Phase 1')
        axs[3].plot(t_phase2 + t1, j_phase2, label='Chaser Phase 2')
        axs[3].plot(t_ball, np.zeros_like(t_ball), '--', color='k', label='Ball jerk = 0 (until t1)')
        axs[3].set_ylabel('Jerk (m/s³)')
        axs[3].set_xlabel('Time (s)')
        axs[3].legend()
        axs[3].grid()
        
        plt.tight_layout()
        plt.show()

    return {
        't_phase1': t_phase1, 'z_phase1': z_phase1, 'vz_phase1': vz_phase1, 'az_phase1': az_phase1, 'j_phase1': j_phase1,
        't_phase2': t_phase2, 'z_phase2': z_phase2, 'vz_phase2': vz_phase2, 'az_phase2': az_phase2, 'j_phase2': j_phase2,
        't_ball': t_ball, 'z_ball': z_ball, 'vz_ball': vz_ball, 'az_ball': az_ball,
    }

In [ ]:
def vd(t, z0, vz0, az0, zc0, vc0, ac0, z_min, t1, t2, t_ball):
    traj = compute_chaser_trajectory(z0, vz0, az0, zc0, vc0, ac0, t1, t2, z_min, plot=False)

    # Ensure arrays are 1D
    t1_vec = traj['t_phase1'].flatten()
    vz1 = np.asarray(traj['vz_phase1']).flatten()

    t2_vec = (traj['t_phase2'] + t1).flatten()
    vz2 = np.asarray(traj['vz_phase2']).flatten()

    # Check that lengths match
    assert t1_vec.shape == vz1.shape, f"t1_vec.shape = {t1_vec.shape}, vz1.shape = {vz1.shape}"
    assert t2_vec.shape == vz2.shape, f"t2_vec.shape = {t2_vec.shape}, vz2.shape = {vz2.shape}"

    # Interpolators
    vz1_interp = interp1d(t1_vec, vz1, kind='cubic', bounds_error=False, fill_value=(vz1[0], vz1[-1]))
    vz2_interp = interp1d(t2_vec, vz2, kind='cubic', bounds_error=False, fill_value=(vz2[0], vz2[-1]))

    if t <= t_ball - t1:
        return np.zeros((3, 1))
    elif t <= t_ball:
        local_t = t - (t_ball - t1)
        vz = vz1_interp(local_t)
        return np.array([[0], [0], [vz]])
    elif t <= t_ball + t2:
        local_t = t - (t_ball - t1)
        vz = vz2_interp(local_t)
        return np.array([[0], [0], [vz]])
    else:
        return np.zeros((3, 1))

In [ ]:
N = 10

m_drone = 10 #kg
g = 9.81

lambda_i = 5.0
flag_ball = [False]

# Ball Tracking
m_ball = 100
z0 = 0
vz0 = 0
az0 = 0

zc0 = 3
ac0 = -9.81

# Ball
v_ball = 0 #velocita iniziale della palla
t_ball = 5.0

z_min = -10  # Minimum height constraint for the chaser

t1_opt, t2_opt = optimize_times_nonlinear(z0, vz0, az0, zc0, v_ball, ac0, z_min)
print(f"Optimized times: t1 = {t1_opt:.3f} s, t2 = {t2_opt:.3f} s")
compute_chaser_trajectory(z0, vz0, az0, zc0, v_ball, ac0, t1_opt, t2_opt, z_min)
t1 = t1_opt
t2 = t2_opt
# Cloath
m_cloth = 10
num_rings = 8
points_per_ring = 20
num_cloth_points = (num_rings-1)*points_per_ring + 1 
m_cloth_per_point = m_cloth/num_cloth_points
k = 2000  # Spring constant for the cloth
damping = 0.5#2 * np.sqrt(k * m_cloth_per_point)  # Damping coefficient for the cloth
cloth = CircularCloth(k=k, damping=damping, m=m_cloth_per_point, num_rings=num_rings, points_per_ring=points_per_ring, num_beams=N, central_index = 0)

V_d = lambda t: vd(t,z0, vz0, az0, zc0, v_ball, ac0, z_min, t1_opt, t2_opt, t_ball)
Gamma_i = np.array([[150, 0, 0], [0, 150, 0], [0, 0, 150]])
masses = np.ones((N,1))*m_drone
L = compute_laplacian_matrix(N, "neighbors") #np.array([[2, -1, -1], [-1, 1, 0], [-1, 0, 1]])

# Add a small random perturbation to the initial positions
np.random.seed(42)  # for reproducibility
perturbation = 0.1 * np.random.randn(3, len(cloth.fixed_indices))
r = np.column_stack([cloth.state0[3*i:3*i+3] for i in cloth.fixed_indices]) + perturbation
print(r)
#r = np.array([[0, 2.5, 0], [2.5*np.sin(np.pi/3), -2.5*np.cos(np.pi/3), 0], [-2.5*np.sin(np.pi/3), -2.5*np.cos(np.pi/3), 0]]).T

initial_condition_drone = np.zeros((6*(N)+N + N))

for i in range(N):
    initial_condition_drone[(i)*6: (i)*6+3] = r[:,i]

initial_condition_drone[-2*N:-N] = np.linspace((m_cloth-1)/N,(m_cloth+1)/N, N) #np.array([0.4, 0.5, 0.6]) #np.ones(3)*0.5
for idx, i in enumerate(range(-N, 0)):
    initial_condition_drone[i] = 0.0#np.arctan2(r[1,idx], r[0,idx])

rho = 100
theta_d = np.array([2*np.pi*i/N for i in range(N)])
controller = Controller(N, Gamma_i, V_d, r, masses, rho = rho)


initial_condition = np.concatenate([initial_condition_drone.flatten(), cloth.state0])

dynamics = total_dynamics(N, m_drone, V_d, Gamma_i, masses, L, r, k, lambda_i, m_ball, v_ball, t_ball, theta_d, controller, cloth)

T_sim = 10
t_vals = np.linspace(0, T_sim, 500)
vd_x, vd_y, vd_z = [], [], []

for t in t_vals:
    v = V_d(t)
    # v is (3,1) or (3,), handle both
    if v.shape == (3, 1):
        v = v.flatten()
    vd_x.append(v[0])
    vd_y.append(v[1])
    vd_z.append(v[2])

plt.figure(figsize=(10, 5))
plt.plot(t_vals, vd_x, label='$v_d^x(t)$')
plt.plot(t_vals, vd_y, label='$v_d^y(t)$')
plt.plot(t_vals, vd_z, label='$v_d^z(t)$')
plt.xlabel('Time [s]')
plt.ylabel('Desired velocity $v_d$ [m/s]')
plt.title('Desired Velocity $v_d(t)$ vs Time')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
t_eval = np.linspace(0, T_sim, 1000)
solution = solve_ivp(dynamics, (0, T_sim), initial_condition, t_eval=t_eval)



In [ ]:
import csv
import numpy as np

def dump_total_state_to_csv(solution, N, cloth, filename='total_state_dump.csv'):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)

        # --- HEADER ---
        headers = ['time']

        # Drone positions and velocities
        for i in range(N):
            headers += [
                f'drone_{i}_x', f'drone_{i}_y', f'drone_{i}_z',
                f'drone_{i}_vx', f'drone_{i}_vy', f'drone_{i}_vz'
            ]

        # Drone masses
        for i in range(N):
            headers += [f'drone_{i}_mass']

        for j in range(cloth.N):  # N = numero nodi
            headers += [
                f'cloth_{j}_x', f'cloth_{j}_y', f'cloth_{j}_z'
            ]

        for j in range(cloth.N):
            headers += [
                f'cloth_{j}_vx', f'cloth_{j}_vy', f'cloth_{j}_vz'
            ]

        writer.writerow(headers)
        print(headers)

        # --- DATA ROWS ---
        for t_idx in range(len(solution.t)):
            row = [solution.t[t_idx]]
            row += solution.y[:, t_idx].tolist()
            writer.writerow(row)

    print(f"✔ Dumped full simulation state to '{filename}'")

dump_total_state_to_csv(solution, N, cloth)

In [ ]:
sol = solution.y[8*N:, :]
time = solution.t

# Plot snapshots
num_plots = 10
indices = np.linspace(0, sol.shape[1]-1, num_plots).astype(int)

fig = plt.figure(figsize=(20, 10))
for k, idx in enumerate(indices):
    ax = fig.add_subplot(2, 5, k+1, projection='3d')
    positions = sol[:3*cloth.N, idx].reshape((cloth.N, 3))


    # Plot all cloth nodes
    ax.scatter(positions[:,0], positions[:,1], positions[:,2], c='blue', s=10)

    # Plot fixed nodes in red
    fixed_idx = cloth.fixed_indices
    if fixed_idx:
        fixed_positions = positions[fixed_idx]
        ax.scatter(fixed_positions[:,0], fixed_positions[:,1], fixed_positions[:,2], c='red', s=30)

    # Draw cloth edges (exclude beams)
    cloth_edge_set = set(cloth.edges) - set(cloth.beam_edges)
    for i, j in cloth_edge_set:
        p1 = positions[i]
        p2 = positions[j]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 'k-', lw=0.7)

    # Draw beams in red
    for i, j in cloth.beam_edges:
        p1 = positions[i]
        p2 = positions[j]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 'r-', lw=2)

    drone_positions = np.array([
        solution.y[6*i : 6*i+3, idx] for i in range(3)
    ])
    ax.scatter(drone_positions[:,0], drone_positions[:,1], drone_positions[:,2],
               c='green', s=50, marker='^', label='Drones')
               
    ax.set_title(f'Time = {time[idx]:.2f}s')
    ax.set_xlim(-cloth.radius*2, cloth.radius*2)
    ax.set_ylim(-cloth.radius*2, cloth.radius*2)
    z_min = positions[:,2].min()
    z_max = positions[:,2].max()
    margin = 0.1 * (z_max - z_min)
    ax.set_zlim(z_min - margin, z_max + margin)

    # Enable XY tickers and labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zticks([])  # Optionally still hide Z-axis ticks

plt.suptitle('Circular Cloth with Fixed Radial Beams Simulation', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
def extract_states(sol, N):
    states = sol.y
    T = states.shape[1]  # number of time steps

    positions = np.zeros((N, 3, T))  # includes center
    velocities = np.zeros((N, 3, T))
    M_values = np.zeros((N, T))
    theta_values = np.zeros((N, T))

    for i in range(N):
        idx = 6 * i
        positions[i] = states[idx : idx + 3]
        
        velocities[i] = states[idx + 3 : idx + 6]

    for i in range(N):
        M_values[i] = states[6 * (N) + i]
    for i in range(N):
        theta_values[i] = states[7 * (N) + i]

    

    return positions, velocities, M_values, theta_values


In [ ]:
positions, velocities, M_values, theta_values = extract_states(solution, N)

# Plot z-position of center and agents
import matplotlib.pyplot as plt

for i in range(N):
    plt.plot(solution.t, positions[i, 2], label=f'Agent {i}')
plt.xlabel('Time [s]')
plt.ylabel('z-position')
plt.legend()
plt.show()



In [ ]:
# Estrai dati di posizione e tempo
time = solution.t
mask = (time >= 0) & (time <= T_sim)

positions = positions[:, :, mask]
time = time[mask]

colors = ['r', 'g', 'b', 'k', 'm', 'c', 'y']

# --- Plot x(t) ---
plt.figure(figsize=(10, 4))
for i in range(N):
    x = positions[i, 0, :]
    plt.plot(time, x, color=colors[i % len(colors)], label=f'Agent {i}')
plt.xlabel('Time [s]')
plt.ylabel('x [m]')
plt.title('X Position vs Time')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# --- Plot y(t) ---
plt.figure(figsize=(10, 4))
for i in range(N):
    y = positions[i, 1, :]
    plt.plot(time, y, color=colors[i % len(colors)], label=f'Agent {i}')
plt.xlabel('Time [s]')
plt.ylabel('y [m]')
plt.title('Y Position vs Time')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Extract Mc_i values

# Time vector from solve_ivp
t = solution.t

# Find indices where time is between 0 and 1
mask = (t >= 0) & (t <= T_sim)
t_filtered = t[mask]
M_filtered = M_values[:, mask]

# Plot
plt.figure(figsize=(10, 6))
for i in range(N):
    plt.plot(t_filtered, M_filtered[i], label=f'$M_{{{i+1}}}(t)$')

plt.xlabel('Time [s]')
plt.ylabel('$M_i$ values')
plt.title('Adaptive Parameter $M_i$ from $t=0$ to $t=1$')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Extract Mc_i values

# Time vector from solve_ivp
t = solution.t

# Find indices where time is between 0 and 1
mask = (t >= 0) & (t <= T_sim)
t_filtered = t[mask]
theta_filtered = theta_values[:, mask]

# Plot
plt.figure(figsize=(10, 6))
for i in range(N):
    plt.plot(t_filtered, theta_filtered[i], label=f'$theta_{{{i+1}}}(t)$')

plt.xlabel('Time [s]')
plt.ylabel('$theta_i$ values')
plt.title('Adaptive Parameter $theta_i$ from $t=0$ to $t=1$')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assume: N = 3 agents (excluding center), so 4 total

# Time range (optional): limit to 0–1 s if needed
mask = (solution.t >= 0) & (solution.t <= T_sim)
positions_xy = positions[:, :2, mask]  # shape: (N+1, 2, T)

# Plot XY trajectories
plt.figure(figsize=(8, 8))
colors = ['r', 'g', 'b', 'k']

for i in range(N):  # 0 = center, 1...N = agents
    x = positions_xy[i, 0, :]
    y = positions_xy[i, 1, :]
    plt.plot(x, y, color=colors[i % len(colors)], label=f'Agent {i}')
    plt.plot(x[0], y[0], 'o', color=colors[i % len(colors)])  # start point
    plt.plot(x[-1], y[-1], 'x', color=colors[i % len(colors)])  # end point

plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('XY Motion of Agents (t = 0 to 1 s)')
plt.grid(True)
plt.legend()
plt.axis('equal')
plt.tight_layout()
plt.show()



In [ ]:
import plotly.graph_objs as go
import numpy as np

# Assume: positions.shape = (N, 3, T)
T = positions.shape[2]
colors = ['red', 'green', 'blue', 'black']

# Initial traces at t=0
data = []
for i in range(N):
    # Trail at t=0 is a single point
    data.append(go.Scatter3d(
        x=[positions[i, 0, 0]],
        y=[positions[i, 1, 0]],
        z=[positions[i, 2, 0]],
        mode='lines+markers',
        name=f'Agent {i}',
        line=dict(color=colors[i % len(colors)], width=4),
        marker=dict(size=4),
        showlegend=True
    ))

# Build frames
frames = []
for t in range(1, T):
    frame_data = []
    for i in range(N):
        # Add trajectory from start to current time
        frame_data.append(go.Scatter3d(
            x=positions[i, 0, :t+1],
            y=positions[i, 1, :t+1],
            z=positions[i, 2, :t+1],
            mode='lines+markers',
            line=dict(color=colors[i % len(colors)], width=4),
            marker=dict(size=4),
            name=f'Agent {i}',
            showlegend=False  # Keep legend only once
        ))
    frames.append(go.Frame(data=frame_data, name=str(t)))

# Axis limits
x_range = [positions[:, 0, :].min(), positions[:, 0, :].max()]
y_range = [positions[:, 1, :].min(), positions[:, 1, :].max()]
z_range = [positions[:, 2, :].min(), positions[:, 2, :].max()]

# Layout
layout = go.Layout(
    title='Animated 3D Motion of Agents',
    scene=dict(
        xaxis=dict(title='x [m]', range=x_range),
        yaxis=dict(title='y [m]', range=y_range),
        zaxis=dict(title='z [m]', range=z_range),
        aspectmode='data',
        camera=dict(
            eye=dict(x=2.0, y=2.0, z=1.0)  # ← Customize this as needed
        )
    ),
    updatemenus=[dict(
        type='buttons',
        showactive=True,
        buttons=[
            dict(label='Play', method='animate', args=[None, {
                'frame': {'duration': 1000 / T, 'redraw': True},
                'fromcurrent': True,
                'transition': {'duration': 0}
            }]),
            dict(label='Pause', method='animate', args=[[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }])
        ]
    )]
)

fig = go.Figure(data=data, layout=layout, frames=frames)
fig.show()

In [ ]:
import plotly.graph_objs as go
import numpy as np

# Assume: positions (N, 3, T), cloth_positions (M, 3, T)
sol = solution.y[8*N:, :]

cloth_positions = sol[:3*cloth.N, :].reshape((cloth.N, 3, -1))

T = positions.shape[2]

# Camera settings
fixed_camera = dict(eye=dict(x=2.0, y=2.0, z=1.0)) 
aspect_ratio_ = dict(x=1, y=1, z=1)
# Axis limits
x_range = [min(positions[:, 0, :].min(), cloth_positions[:, 0, :].min()),
           max(positions[:, 0, :].max(), cloth_positions[:, 0, :].max())]
y_range = [min(positions[:, 1, :].min(), cloth_positions[:, 1, :].min()),
           max(positions[:, 1, :].max(), cloth_positions[:, 1, :].max())]
z_range = [min(positions[:, 2, :].min(), cloth_positions[:, 2, :].min()),
           max(positions[:, 2, :].max(), cloth_positions[:, 2, :].max()) + 3] # TODO:

colors = ['red', 'green', 'blue', 'black']

# Initial traces at t=0
data = []

# --- Drones ---
for i in range(N):
    data.append(go.Scatter3d(
        x=[positions[i, 0, 0]],
        y=[positions[i, 1, 0]],
        z=[positions[i, 2, 0]],
        mode='markers+text',
        name=f'Agent {i}',
        marker=dict(size=6, color=colors[i % len(colors)]),
        text=[f'{i}'],
        textposition='top center',
        showlegend=True
    ))

# --- Cloth Nodes (blue) ---
data.append(go.Scatter3d(
    x=cloth_positions[:, 0, 0],
    y=cloth_positions[:, 1, 0],
    z=cloth_positions[:, 2, 0],
    mode='markers',
    marker=dict(size=3, color='blue'),
    name='Cloth Nodes',
    showlegend=True
))

# --- Fixed Nodes (red) ---
fixed_idx = cloth.fixed_indices
if fixed_idx:
    fixed_positions = cloth_positions[fixed_idx, :, 0]
    data.append(go.Scatter3d(
        x=fixed_positions[:, 0],
        y=fixed_positions[:, 1],
        z=fixed_positions[:, 2],
        mode='markers',
        marker=dict(size=5, color='red'),
        name='Fixed Nodes',
        showlegend=True
    ))

# --- Cloth Edges ---
edge_lines = []
for i, j in cloth.edges:
    p1 = cloth_positions[i, :, 0]
    p2 = cloth_positions[j, :, 0]
    edge_lines.extend([p1, p2, [None, None, None]])
edge_lines = np.array(edge_lines).T

data.append(go.Scatter3d(
    x=edge_lines[0],
    y=edge_lines[1],
    z=edge_lines[2],
    mode='lines',
    line=dict(color='gray', width=1),
    name='Cloth Edges',
    showlegend=False
))

# --- Tempo iniziale (t = 0) ---
data.append(go.Scatter3d(
    x=[x_range[0]],
    y=[y_range[1]],
    z=[z_range[1]],
    mode='text',
    text=[f't = 0.00 s'],
    showlegend=False,
    textfont=dict(size=12, color='black')
))


# --- FRAMES ---
frames = []



# Palla: traiettoria verticale verso cloth.central_index
t_index = np.searchsorted(solution.t, t_ball)
z_impact = cloth_positions[cloth.central_index, 2, t_index]
x_impact = cloth_positions[cloth.central_index, 0, t_index]
y_impact = cloth_positions[cloth.central_index, 1, t_index]

from collections import deque

ball_positions = deque()
#ball_positions.append([x_impact, y_impact, z_impact])  # t_index


# Trova quando la palla inizia a cadere
t_start = t_ball - t1

# Trova l'indice del tempo di inizio caduta
t_start_index = np.searchsorted(solution.t, t_start)

# Inizializza la posizione iniziale della palla (altezza da cui parte)
z0_ball = z_impact - v_ball * t1 - 0.5 * (-9.81) * t1**2  # da moto uniformemente accelerato
z_curr = z0_ball

for t_idx in range(0, t_start_index):
    ball_positions.append([x_impact, y_impact, 10])
# --- PRIMA dell’impatto: traiettoria parabolica ---
for t_idx in range(t_start_index, t_index):
    dt = solution.t[t_idx] - solution.t[t_start_index]
    z = z0_ball + v_ball * dt - 0.5 * 9.81 * dt**2
    ball_positions.append([x_impact, y_impact, z])

# --- IMPATTO ---
ball_positions.append([x_impact, y_impact, z_impact])  # t_index

# --- DOPO l’impatto: la palla segue il nodo centrale della cloth ---
for t_idx in range(t_index + 1, len(solution.t)):
    z_post = cloth_positions[cloth.central_index, 2, t_idx]
    x_post = cloth_positions[cloth.central_index, 0, t_idx]
    y_post = cloth_positions[cloth.central_index, 1, t_idx]
    ball_positions.append([x_post, y_post, z_post])

ball_positions = np.array(ball_positions)

for t in range(1, min(T, len(ball_positions))):
    frame_data = []

    # Drones
    for i in range(N):
        frame_data.append(go.Scatter3d(
            x=[positions[i, 0, t]],
            y=[positions[i, 1, t]],
            z=[positions[i, 2, t]],
            mode='markers',
            marker=dict(size=6, color=colors[i % len(colors)]),
            name=f'Agent {i}',
            showlegend=False
        ))

    # Cloth nodes (blue)
    frame_data.append(go.Scatter3d(
        x=cloth_positions[:, 0, t],
        y=cloth_positions[:, 1, t],
        z=cloth_positions[:, 2, t],
        mode='markers',
        marker=dict(size=3, color='blue'),
        name='Cloth Nodes',
        showlegend=False
    ))

    # --- Palla ---
    ball_xyz = ball_positions[t]
    frame_data.append(go.Scatter3d(
        x=[ball_xyz[0]],
        y=[ball_xyz[1]],
        z=[ball_xyz[2]],
        mode='markers',
        marker=dict(size=6, color='orange', symbol='circle'),
        name='Ball',
        showlegend=(t == 1)
    ))

    # Fixed nodes (red)
    if fixed_idx:
        fixed_positions = cloth_positions[fixed_idx, :, t]
        frame_data.append(go.Scatter3d(
            x=fixed_positions[:, 0],
            y=fixed_positions[:, 1],
            z=fixed_positions[:, 2],
            mode='markers',
            marker=dict(size=5, color='red'),
            name='Fixed Nodes',
            showlegend=False
        ))

    # Cloth edges (gray)
    edge_lines = []
    for i, j in cloth.edges:
        p1 = cloth_positions[i, :, t]
        p2 = cloth_positions[j, :, t]
        edge_lines.extend([p1, p2, [None, None, None]])
    edge_lines = np.array(edge_lines).T

    frame_data.append(go.Scatter3d(
        x=edge_lines[0],
        y=edge_lines[1],
        z=edge_lines[2],
        mode='lines',
        line=dict(color='gray', width=1),
        name='Cloth Edges',
        showlegend=False
    ))

    # Tempo corrente in alto
    frame_data.append(go.Scatter3d(
        x=[x_range[0]],
        y=[y_range[1]],
        z=[z_range[1]],
        mode='text',
        text=[f't = {solution.t[t]:.2f} s'],
        showlegend=False,
        textfont=dict(size=12, color='black')
    ))

    frames.append(go.Frame(
    data=frame_data,
    name=str(t),
    layout=go.Layout(
        scene=dict(
            xaxis=dict(range=x_range),
            yaxis=dict(range=y_range),
            zaxis=dict(range=z_range),
            camera=fixed_camera,
            aspectmode='manual',
            aspectratio=aspect_ratio_  # O adatta z in base alla tua scena
        ),
        annotations=[
            dict(
                text=f"<b>t = {solution.t[t]:.2f} s</b>",
                showarrow=False,
                x=0.5,
                y=1.08,
                xref="paper",
                yref="paper",
                font=dict(size=16, color="black"),
                align="center"
            )
        ]
    )
))




# Layout
layout = go.Layout(
    width=1000,  # 👈 Larghezza in pixel
    height=800,  # 👈 Altezza in pixel
    title='Animated 3D Motion: Drones and Cloth',
    scene=dict(
        xaxis=dict(title='x [m]', range=x_range),
        yaxis=dict(title='y [m]', range=y_range),
        zaxis=dict(title='z [m]', range=z_range),
        #aspectmode='data',
        aspectmode='manual',
        aspectratio=aspect_ratio_,  # O adatta z in base alla tua scena
        camera=fixed_camera  # Wide initial view
    ),
    updatemenus=[dict(
        type='buttons',
        showactive=True,
        buttons=[
            dict(label='Play', method='animate', args=[None, {
                'frame': {'duration': 10, 'redraw': True},  # ← Slower animation
                'fromcurrent': True,
                'transition': {'duration': 0}
            }]),
            dict(label='Pause', method='animate', args=[[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }])
        ]
    )],
    annotations=[
        dict(
            text="<b>t = 0.00 s</b>",
            showarrow=False,
            x=0.5,
            y=1.08,
            xref="paper",
            yref="paper",
            font=dict(size=16, color="black"),
            align="center"
        )
    ]
)

# Create figure and show
fig = go.Figure(data=data, layout=layout, frames=frames)
fig.show()

fig.write_html("animated_drones_cloth.html")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Mask to filter time
mask = (solution.t >= 0) & (solution.t <= T_sim)
t_filtered = solution.t[mask]
velocities_filtered = velocities[:, :, mask]  # shape: (N, 3, T_filtered)

components = ['x', 'y', 'z']
colors = ['r', 'g', 'b', 'k', 'orange', 'purple', 'cyan']

V_d = lambda t: vd(t,z0, vz0, az0, zc0, v_ball, ac0, z_min, t1_opt, t2_opt, t_ball)
print(z0, vz0, az0, zc0, v_ball, ac0, z_min, t1_opt, t2_opt, t_ball)
vd_x, vd_y, vd_z = [], [], []

for t in t_filtered:
    v = V_d(t)
    # v is (3,1) or (3,), handle both
    if v.shape == (3, 1):
        v = v.flatten()
    vd_x.append(v[0])
    vd_y.append(v[1])
    vd_z.append(v[2])

plt.figure(figsize=(10, 4))
plt.plot(t_filtered, vd_z, label='$v_d^z(t)$')

for dim in range(3):  # 0=x, 1=y, 2=z
    plt.figure(figsize=(10, 4))
    if dim ==0:
        plt.plot(t_filtered, vd_x, label='$v_d^x(t)$')
    elif dim == 1:
        plt.plot(t_filtered, vd_y, label='$v_d^y(t)$')
    else:
        plt.plot(t_filtered, vd_z, label='$v_d^z(t)$')
    for i in range(N):
        plt.plot(t_filtered, velocities_filtered[i, dim, :], label=f'Agent {i}', color=colors[i % len(colors)])

    plt.title(f'Velocity in {components[dim]} direction (0–1 s)')
    plt.xlabel('Time [s]')
    plt.ylabel(f'$v_{components[dim]}$ [m/s]')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV log
df = pd.read_csv('control_log.csv')

# Extract time
time = df['time']

# Identify how many agents by parsing columns
agent_ids = sorted(set(col.split('_')[0] for col in df.columns if col != 'time'))

# Plot control inputs for each agent
for agent_id in agent_ids:
    plt.figure(figsize=(10, 4))
    plt.plot(time, df[f'{agent_id}_x'], label='x')
    plt.plot(time, df[f'{agent_id}_y'], label='y')
    plt.plot(time, df[f'{agent_id}_z'], label='z')
    plt.title(f'Control Input for {agent_id.upper()}')
    plt.xlabel('Time [s]')
    plt.ylabel('Control Force')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()



In [ ]:
import numpy as np

# Loop over agents and plot the norm of control input
for agent_id in agent_ids:
    # Extract control components
    ux = df[f'{agent_id}_x']
    uy = df[f'{agent_id}_y']
    uz = df[f'{agent_id}_z']

    # Compute control norm (Euclidean norm)
    u_norm = np.sqrt(ux**2 + uy**2 + uz**2)

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(time, u_norm, label='‖u‖ [N]', color='blue')
    plt.axhline(900, color='red', linestyle='--', label='Max Safe Limit (900 N)')
    plt.title(f'Control Input Magnitude for {agent_id.upper()}')
    plt.xlabel('Time [s]')
    plt.ylabel('‖u‖ [N]')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    # Calcola il massimo della norma del controllo su tutti i droni e tutti i timestep
    max_u_norm = 0
    for agent_id in agent_ids:
        ux = df[f'{agent_id}_x']
        uy = df[f'{agent_id}_y']
        uz = df[f'{agent_id}_z']
        u_norm = np.sqrt(ux**2 + uy**2 + uz**2)
        max_u_norm = max(max_u_norm, np.max(u_norm))
    print(f"Max control input norm across all agents and timesteps: {max_u_norm:.2f} N")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define V_d(t)
V_d = lambda t: (1 - np.exp(-t)) * np.array([2.0, 1.0, 0.0])

# Time vector
t_vals = np.linspace(0, 5, 500)
Vx = []
Vy = []
Vz = []

# Compute components over time
for t in t_vals:
    v = V_d(t)
    Vx.append(v[0])
    Vy.append(v[1])
    Vz.append(v[2])

# Plot
plt.figure(figsize=(10, 5))
plt.plot(t_vals, Vx, label='Vx(t)')
plt.plot(t_vals, Vy, label='Vy(t)')
plt.plot(t_vals, Vz, label='Vz(t)')
plt.title('Desired Velocity V_d(t)')
plt.xlabel('Time [s]')
plt.ylabel('Velocity [m/s]')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the logged data
df_control = pd.read_csv('force_log.csv')   # control forces
df_f_hat = pd.read_csv('f_hat_log.csv')         # estimated internal forces

# Extract time and number of agents
time = df_control['time']
#N = (df_control.shape[1] - 1) // 3
components = ['x', 'y', 'z']

# Plot each component for each agent
fig, axs = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

for i, comp in enumerate(components):
    for agent in range(N):
        # Plot actual control forces
        print(time.shape, df_control[f'f{agent}_{comp}'].shape, df_f_hat[f'f{agent}_{comp}'].shape)
        axs[i].plot(time, df_control[f'f{agent}_{comp}'], label=f'Agent {agent} f_{comp} (control)', linestyle='-')
        # Plot estimated forces
        axs[i].plot(time, df_f_hat[f'f{agent}_{comp}'], label=f'Agent {agent} f_hat_{comp}', linestyle='--')
    
    axs[i].set_ylabel(f'$f_{comp}$ [N]')
    axs[i].grid(True)
    axs[i].legend(loc='upper right')

axs[2].set_xlabel('Time [s]')
fig.suptitle('Control Forces vs Estimated Internal Forces $f_i$ Over Time')
plt.tight_layout()
plt.show()




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the data
df_control = pd.read_csv('force_log.csv')
df_f_hat = pd.read_csv('f_hat_log.csv')

# Extract time and number of agents
time = df_control['time']
N = (df_control.shape[1] - 1) // 3
components = ['x', 'y', 'z']

# Compute 2-norm of the difference between control forces and estimated forces
for agent in range(N):
    diff_sq = np.zeros(len(time))
    for comp in components:
        f = df_control[f'f{agent}_{comp}']
        f_hat = df_f_hat[f'f{agent}_{comp}']
        diff_sq += (f - f_hat)**2

    norm = np.sqrt(diff_sq)

    # Create a separate plot for each agent
    plt.figure(figsize=(8, 4))
    plt.plot(time, norm, label=f'Agent {agent}')
    plt.title(f'2-Norm of Force Difference for Agent {agent}')
    plt.xlabel('Time [s]')
    plt.ylabel('$\\|f_i - \\hat{{f}}_i\\|_2$ [N]')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()




In [ ]:
%pip install openpyxl
import pandas as pd
#risultati ottenuti con cloth e droni di 10kg ciascuno, per ottenere 10 metri di vertical clearance

# Load the Excel file
df = pd.read_excel("mass_vs_N.xlsx", index_col=0)

# Ensure columns are sorted (N_drones)
df = df.sort_index(axis=1)

print(df.head())  

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Crea una colormap personalizzata da verde a rosso passando per giallo
# Soglia di sicurezza
threshold = 1000

# Range dei valori
vmin = df.min().min()
vmax = df.max().max()

# Normalizza con attenzione alla soglia (per forzare il rosso oltre 1000)
norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=threshold, vmax=vmax)

# Definizione colormap con cambio netto oltre soglia
cmap = mcolors.LinearSegmentedColormap.from_list(
    "sharp_green_yellow_red",
    [
        (0.0, "green"),
        (0.4, "yellow"),
        (1.0, "red")
    ]
)

plt.figure(figsize=(10, 6))
sns.heatmap(df,
            annot=True,
            fmt=".1f",
            cmap=cmap,
            norm=norm,
            linewidths=0.5,
            linecolor='gray',
            cbar_kws={'label': 'Control Input [N]'})

plt.title("Control Input Heatmap vs. Number of Drones and Payload Mass")
plt.xlabel("Number of Drones")
plt.ylabel("Payload Mass [kg]")
plt.tight_layout()
plt.show()



In [ ]:
for idx, row in df.iterrows():
    plt.plot(row.index, row.values, marker='o', label=f'{idx} kg')

plt.axhline(1000, color='red', linestyle='--', label='Actuator Limit (900 N)')
plt.title("Control Input vs Number of Drones")
plt.xlabel("Number of Drones")
plt.ylabel("Control Input [N]")
plt.grid(True)
plt.legend(title="Payload Mass")
plt.tight_layout()
plt.show()